# 🔁 Fibonacci Jackpot Reversal Detector with XGBoost Filtering
- Scans all NSE Futures daily
- Filters signals using trained XGBoost model
- Sends Telegram alert if confidence > 75%
- Avoids duplicate alerts


In [ ]:
!pip install -q yfinance ta joblib


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import joblib
import requests
import os
from datetime import datetime
from ta.momentum import RSIIndicator
from ta.trend import MACD, ADXIndicator


In [ ]:
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID")
MODEL_PATH = "fibonacci_xgb_model.pkl"

model = joblib.load(MODEL_PATH)
print("✅ XGBoost model loaded.")


In [ ]:
def send_telegram_message(message):
    if TELEGRAM_BOT_TOKEN and TELEGRAM_CHAT_ID:
        url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        payload = {"chat_id": TELEGRAM_CHAT_ID, "text": message}
        response = requests.post(url, data=payload)
        print("✅ Telegram alert sent" if response.ok else "❌ Telegram error:", response.text)
    else:
        print("⚠️ Telegram credentials not set.")


In [ ]:
df_symbols = pd.read_csv("futures_list.csv")
nifty_futures = df_symbols["Symbol"].dropna().unique().tolist()
print(f"Loaded {len(nifty_futures)} futures symbols.")


In [ ]:
log_path = "signal_log.csv"
today = datetime.today().strftime('%Y-%m-%d')
if os.path.exists(log_path):
    sent_log = pd.read_csv(log_path)
else:
    sent_log = pd.DataFrame(columns=["date", "symbol"])


In [ ]:
results = []
features = ["RSI", "MACD_Hist", "ADX", "Volume_Change"]

for symbol in nifty_futures:
    try:
        df = yf.download(f"{symbol}.NS", period="6mo", interval="1d", progress=False)
        if df.empty:
            continue

        df["RSI"] = RSIIndicator(df["Close"]).rsi()
        macd = MACD(df["Close"])
        df["MACD_Hist"] = macd.macd_diff()
        df["ADX"] = ADXIndicator(df["High"], df["Low"], df["Close"]).adx()
        df["Volume_Change"] = df["Volume"].pct_change() * 100

        high = df['High'].rolling(10).max()
        low = df['Low'].rolling(10).min()
        retracement = high - (high - low) * 0.618
        df["near_618"] = df["Low"] <= retracement

        latest = df.iloc[-1]
        if latest["near_618"]:
            row_features = latest[features].fillna(0).values.reshape(1, -1)
            confidence = model.predict_proba(row_features)[0][1]
            if confidence > 0.75:
                already_sent = ((sent_log["date"] == today) & (sent_log["symbol"] == symbol)).any()
                if not already_sent:
                    message = (
                        f"🚨 AI Fibonacci Reversal Alert: {symbol}
"
                        f"Date: {latest.name.date()}
"
                        f"Confidence: {round(confidence * 100, 2)}%
"
                        f"Close: ₹{round(latest['Close'], 2)}"
                    )
                    send_telegram_message(message)
                    sent_log = pd.concat([sent_log, pd.DataFrame([{"date": today, "symbol": symbol}])])
                    results.append({
                        "Symbol": symbol,
                        "Confidence": round(confidence, 3),
                        "Close": round(latest["Close"], 2)
                    })
    except Exception as e:
        print(f"❌ Error with {symbol}: {e}")

sent_log.to_csv(log_path, index=False)


In [ ]:
pd.DataFrame(results)
